In [193]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [194]:
df = pd.read_csv('/Users/rajivkumar/Desktop/Carbon Emission/Carbon Emission.csv')
df.head(5)

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,Recycling,Cooking_With,CarbonEmission
0,overweight,female,pescatarian,daily,coal,public,NaN,often,230,frequently,210,large,4,7,26,1,No,['Metal'],"['Stove', 'Oven']",2238
1,obese,female,vegetarian,less frequently,natural gas,walk/bicycle,NaN,often,114,rarely,9,extra large,3,9,38,5,No,['Metal'],"['Stove', 'Microwave']",1892
2,overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,2472,small,1,14,47,6,Sometimes,['Metal'],"['Oven', 'Microwave']",2595
3,overweight,male,omnivore,twice a day,wood,walk/bicycle,NaN,sometimes,157,rarely,74,medium,3,20,5,7,Sometimes,"['Paper', 'Plastic', 'Glass', 'Metal']","['Microwave', 'Grill', 'Airfryer']",1074
4,obese,female,vegetarian,daily,coal,private,diesel,often,266,very frequently,8457,large,1,3,5,6,Yes,['Paper'],['Oven'],4743


In [195]:
df.shape

(10000, 20)

Handling Null Value

In [196]:
df.isnull().sum()

Body Type                           0
Sex                                 0
Diet                                0
How Often Shower                    0
Heating Energy Source               0
Transport                           0
Vehicle Type                     6721
Social Activity                     0
Monthly Grocery Bill                0
Frequency of Traveling by Air       0
Vehicle Monthly Distance Km         0
Waste Bag Size                      0
Waste Bag Weekly Count              0
How Long TV PC Daily Hour           0
How Many New Clothes Monthly        0
How Long Internet Daily Hour        0
Energy efficiency                   0
Recycling                           0
Cooking_With                        0
CarbonEmission                      0
dtype: int64

In [197]:
df["Vehicle Type"] = df.apply(
    lambda row: "Other" if (pd.isna(row["Vehicle Type"]) and row["Transport"] == "private")
    else ("None" if pd.isna(row["Vehicle Type"]) else row["Vehicle Type"]),
    axis=1
)

In [198]:
df.isnull().sum()

Body Type                        0
Sex                              0
Diet                             0
How Often Shower                 0
Heating Energy Source            0
Transport                        0
Vehicle Type                     0
Social Activity                  0
Monthly Grocery Bill             0
Frequency of Traveling by Air    0
Vehicle Monthly Distance Km      0
Waste Bag Size                   0
Waste Bag Weekly Count           0
How Long TV PC Daily Hour        0
How Many New Clothes Monthly     0
How Long Internet Daily Hour     0
Energy efficiency                0
Recycling                        0
Cooking_With                     0
CarbonEmission                   0
dtype: int64

In [199]:
df.head(3)

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,Recycling,Cooking_With,CarbonEmission
0,overweight,female,pescatarian,daily,coal,public,None,often,230,frequently,210,large,4,7,26,1,No,['Metal'],"['Stove', 'Oven']",2238
1,obese,female,vegetarian,less frequently,natural gas,walk/bicycle,None,often,114,rarely,9,extra large,3,9,38,5,No,['Metal'],"['Stove', 'Microwave']",1892
2,overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,2472,small,1,14,47,6,Sometimes,['Metal'],"['Oven', 'Microwave']",2595


Handling Outlier

In [200]:
df.describe()

,Monthly Grocery Bill,Vehicle Monthly Distance Km,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,CarbonEmission
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,173.875200,2031.485900,4.024600,12.139200,25.109000,11.889100,2269.147300
std,72.234018,2769.715597,1.990375,7.106369,14.698725,7.277218,1017.675247
min,50.000000,0.000000,1.000000,0.000000,0.000000,0.000000,306.000000
25%,111.000000,69.000000,2.000000,6.000000,13.000000,6.000000,1538.000000
50%,173.000000,823.000000,4.000000,12.000000,25.000000,12.000000,2080.000000
75%,237.000000,2516.750000,6.000000,18.000000,38.000000,18.000000,2768.000000
max,299.000000,9999.000000,7.000000,24.000000,50.000000,24.000000,8377.000000


In [201]:
numerical_cols = df.select_dtypes(include='int64').columns.tolist()
numerical_cols.remove('CarbonEmission')

def remove_outliers_iqr(df, cols):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df


df = remove_outliers_iqr(df, numerical_cols)

In [202]:
df.describe()

,Monthly Grocery Bill,Vehicle Monthly Distance Km,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,CarbonEmission
count,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000
mean,173.411900,1122.027912,4.020675,12.119113,25.102573,11.897772,2063.876637
std,72.089313,1497.026416,1.991489,7.104209,14.698413,7.270636,781.289904
min,50.000000,0.000000,1.000000,0.000000,0.000000,0.000000,306.000000
25%,111.000000,61.000000,2.000000,6.000000,13.000000,6.000000,1482.000000
50%,172.000000,479.000000,4.000000,12.000000,25.000000,12.000000,1965.000000
75%,236.000000,1609.500000,6.000000,18.000000,38.000000,18.000000,2544.000000
max,299.000000,6188.000000,7.000000,24.000000,50.000000,24.000000,6260.000000


In [203]:
for col in ["Recycling", "Cooking_With"]:
    df[col] = df[col].str.strip("[]").str.replace("'", "").str.split(", ")
    dummies = df[col].str.join('|').str.get_dummies()
    dummies = dummies.add_prefix(col + "_")
    df = pd.concat([df.drop(columns=[col]), dummies], axis=1)

In [204]:
df.head(3)

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,...,CarbonEmission,Recycling_Glass,Recycling_Metal,Recycling_Paper,Recycling_Plastic,Cooking_With_Airfryer,Cooking_With_Grill,Cooking_With_Microwave,Cooking_With_Oven,Cooking_With_Stove
0,overweight,female,pescatarian,daily,coal,public,None,often,230,frequently,...,2238,0,1,0,0,0,0,0,1,1
1,obese,female,vegetarian,less frequently,natural gas,walk/bicycle,None,often,114,rarely,...,1892,0,1,0,0,0,0,1,0,1
2,overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,...,2595,0,1,0,0,0,0,1,1,0


In [205]:
df.shape

(8706, 27)

In [206]:
df.select_dtypes(include='object').columns

Index(['Body Type', 'Sex', 'Diet', 'How Often Shower', 'Heating Energy Source',
       'Transport', 'Vehicle Type', 'Social Activity',
       'Frequency of Traveling by Air', 'Waste Bag Size', 'Energy efficiency'],
      dtype='object')

In [207]:
df.describe()

,Monthly Grocery Bill,Vehicle Monthly Distance Km,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,CarbonEmission,Recycling_Glass,Recycling_Metal,Recycling_Paper,Recycling_Plastic,Cooking_With_Airfryer,Cooking_With_Grill,Cooking_With_Microwave,Cooking_With_Oven,Cooking_With_Stove
count,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000,8706.000000
mean,173.411900,1122.027912,4.020675,12.119113,25.102573,11.897772,2063.876637,0.500574,0.507007,0.499311,0.500804,0.499426,0.499426,0.508040,0.504020,0.506317
std,72.089313,1497.026416,1.991489,7.104209,14.698413,7.270636,781.289904,0.500028,0.499980,0.500028,0.500028,0.500028,0.500028,0.499964,0.500013,0.499989
min,50.000000,0.000000,1.000000,0.000000,0.000000,0.000000,306.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,111.000000,61.000000,2.000000,6.000000,13.000000,6.000000,1482.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,172.000000,479.000000,4.000000,12.000000,25.000000,12.000000,1965.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
75%,236.000000,1609.500000,6.000000,18.000000,38.000000,18.000000,2544.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,299.000000,6188.000000,7.000000,24.000000,50.000000,24.000000,6260.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [208]:
x = df.drop('CarbonEmission', axis=1)
y = df['CarbonEmission']

In [209]:
from sklearn.model_selection import train_test_split

In [210]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=50)

In [211]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib

In [212]:
categorical_features = x.select_dtypes(include=['object', 'category']).columns
numerical_features = x.select_dtypes(include=np.number).columns

In [213]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [214]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

In [215]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])

pipeline.fit(x_train, y_train)
joblib.dump(pipeline, "carbon_pipeline.joblib")

['carbon_pipeline.joblib']

In [216]:
pipeline = joblib.load("carbon_pipeline.joblib")
y_pred = pipeline.predict(x_test)

In [217]:
from sklearn.metrics import mean_absolute_error, r2_score

In [218]:
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [219]:
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R^2 Score: {r2:.2f}")

Mean Absolute Error: 188.25
R^2 Score: 0.90
